<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/chatbot_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !rm -rf /content/Poly-Encoder

In [1]:
!nvidia-smi

Sat Jul  9 04:20:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q transformers datasets folium==0.2.1

     |████████████████████████████████| 4.4 MB 7.5 MB/s 
     |████████████████████████████████| 362 kB 69.3 MB/s 
     |████████████████████████████████| 69 kB 8.6 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 6.6 MB 66.4 MB/s 
     |████████████████████████████████| 596 kB 88.0 MB/s 
     |████████████████████████████████| 1.1 MB 72.3 MB/s 
     |████████████████████████████████| 212 kB 78.9 MB/s 
     |████████████████████████████████| 140 kB 84.6 MB/s 
     |████████████████████████████████| 127 kB 77.6 MB/s 
     |████████████████████████████████| 144 kB 87.6 MB/s 
     |████████████████████████████████| 271 kB 72.1 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 


In [4]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git
%cd /content/Poly-Encoder

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 129 (delta 41), reused 20 (delta 8), pack-reused 57
Receiving objects: 100% (129/129), 40.70 KiB | 8.14 MiB/s, done.
Resolving deltas: 100% (66/66), done.
/content/Poly-Encoder


In [5]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG
!gdown -q --folder 1RH7laK4WlucCw68ZeExFvyg7vs-kB_x3

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder
from transform import SelectionJoinTransform, SelectionSequentialTransform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

PATH = '/content/Poly-Encoder/chatbot_output/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH))
model.to(device)
model.device

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device(type='cuda', index=0)

In [7]:
context = ['This framework generates embeddings for each input sentence', 
            'Sentences are passed as a list of string.', 
            'The quick brown fox jumps over the lazy dog.']

candidates = ['This framework generates embeddings for each input sentence', 
            'Sentences are passed as a list of string.', 
            'The quick brown fox jumps over the lazy dog.']

In [8]:
def context_input(context):
    context_input_ids, context_input_masks = context_transform(context)
    contexts_token_ids_list_batch, contexts_input_masks_list_batch = [context_input_ids], [context_input_masks]

    long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch]

    contexts_token_ids_list_batch, contexts_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return contexts_token_ids_list_batch, contexts_input_masks_list_batch

In [9]:
contexts_token_ids_list_batch, contexts_input_masks_list_batch = context_input(context)

In [43]:
def response_input(candidates):
    responses_token_ids_list, responses_input_masks_list = response_transform(candidates)
    responses_token_ids_list_batch, responses_input_masks_list_batch = [responses_token_ids_list], [responses_input_masks_list]

    long_tensors = [responses_token_ids_list_batch, responses_input_masks_list_batch]

    responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return responses_token_ids_list_batch, responses_input_masks_list_batch

In [11]:
responses_token_ids_list_batch, responses_input_masks_list_batch = response_input(candidates)

In [12]:
def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
        
        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

In [13]:
embs = embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch)

In [14]:
def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
                
        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

In [15]:
cand_emb = cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch)

In [16]:
cand_emb

tensor([[[ 0.2946, -0.5343,  0.4598,  ..., -0.3450,  0.1062,  0.9311],
         [ 0.0754, -0.3877,  0.4182,  ..., -0.3769,  0.1252,  0.8174],
         [-0.0020, -0.2832,  0.5391,  ..., -0.5368, -0.0037,  0.7433]]],
       device='cuda:0')

In [17]:
def score(embs, cand_emb):
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

In [18]:
score(embs, cand_emb)

tensor([[498.3091, 500.8978, 504.1481]], device='cuda:0')

In [19]:
# forward
with torch.no_grad():
    model.eval()
    
    model_foward = model(contexts_token_ids_list_batch, contexts_input_masks_list_batch, responses_token_ids_list_batch, responses_input_masks_list_batch)
    print(model_foward)

tensor([[498.3091, 500.8978, 504.1481]], device='cuda:0')


### 데이터 검증

In [20]:
import pickle

with open('/content/Poly-Encoder/감성대화챗봇데이터/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)
with open('/content/Poly-Encoder/감성대화챗봇데이터/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [21]:
# index = 500

In [22]:
# train[index]['context']

In [23]:
# train[index]['responses']

In [24]:
# dev[index]['context']

In [25]:
# dev[index]['responses']

### 챗봇 테이블 생성

In [26]:
data = {
    'context' : [],
    'response': []
}

In [27]:
for sample in train:
    data['context'].append(sample['context'])
    data['response'].append([sample['responses'][0]])

In [28]:
len(data['context']), len(data['response'])

(40879, 40879)

In [29]:
idx = 400

In [30]:
print(data['context'][idx])
print(data['response'][idx])

['participant 1: 성적이 오르지 않아 고민이었는데 담임 선생님께서 도움을 많이 주셔서 매우 감사해.', 'participant 2: 담임 선생님께서 도움을 많이 주셨군요. 감사한 마음을 어떻게 표현하면 좋을까요?', 'participant 1: 글쎄. 어떻게 전해야 할지 몰라서 정말 고민이야.']
['감사를 표할 방법에 대해 고민되시나 보네요.']


In [31]:
import pandas as pd

In [32]:
df = pd.DataFrame(data)

### generate cand_embs & create tensor table

In [48]:
# Faster way to load..?

# responses = []
# for idx in range(len(df)):
#     responses.append(df['response'][idx][0])
# cands = response_input(responses)
# cand_embs = cand_emb_gen(*cands).to('cpu')

In [72]:
response_input_srs = df['response'].apply(response_input)

In [73]:
response_input_lst = response_input_srs.to_list()

In [74]:
cand_embs_lst = []
for sample in response_input_lst:
    cand_embs_lst.append(cand_emb_gen(*sample).to('cpu'))

In [75]:
cand_embs = cand_embs_lst[0]
for idx in range(1, len(cand_embs_lst)):
    y = cand_embs_lst[idx]
    cand_embs = torch.cat((cand_embs, y), 1)

In [76]:
cand_embs = cand_embs.to(device)

In [77]:
cand_embs.shape

torch.Size([1, 40879, 768])

### generate context_embs

In [208]:
query = ['너무 성급한 결정을 한 것 같아.']

In [209]:
embs = embs_gen(*context_input(query))

### Score & Retrieve

In [210]:
s = score(embs, cand_embs)

In [211]:
idx = s.argmax(1)

In [212]:
idx = int(idx[0])

In [213]:
df['response'][idx]

['섣부른 결정에 후회하고 계시는군요. 어떻게 하고 싶으세요?']

In [214]:
df.iloc[idx]['context']

['participant 1: 가정 형편에 맞는 차를 계약하려 했는데 자동차 판매사원 판매 전략에 넘어가 버렸어.',
 'participant 2: 예산을 초과하는 금액을 지출하게 되셨군요.',
 'participant 1: 소나타 차량을 생각하고 갔는데 신형 그랜저에 앉아 보고서 마음이 변했지 뭐야.',
 'participant 2: 좋은 걸 사고 싶은 마음과 재정 걱정에 혼란스러우셨겠어요.',
 'participant 1: 집에 와서 다시 생각해보니 잘못 결정한 것 같아.']